<div style="width:100%;text-align: center;"> <img align=middle src="https://c.tenor.com/DCNcMv2gPnIAAAAM/heart-attack-heartbeat.gif" alt="Heat beating" style="height:366px;margin-top:3rem;"> </div>

# <h1 style='background:#E64848; border:0; color:white'><center>🩺♥Heart Attack Analysis & Prediction</center></h1> 

# **<span style="color:#FEB139;">📰Get the Data</span>**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import mean_absolute_error, accuracy_score, roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

import imblearn
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

#models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline

#metrics
from imblearn.over_sampling import SMOTE
from numpy import mean
from scipy.stats import uniform
from scipy import interp
from sklearn.utils import class_weight

In [ ]:
heart_data = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
O2Saturation_data = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')

# **<span style="color:#FEB139;">📄About the Dataset</span>**

This dataset contains 14 columns:

**Age** : Age of the patient

**Sex** : Sex of the patient

**exang**: exercise induced angina 

> 1 = yes

> 0 = no

**ca**: number of major vessels (0-3)

**cp** : Chest Pain type chest pain type

> *Value 1*: typical angina

> *Value 2*: atypical angina

> *Value 3*: non-anginal pain

> *Value 4*: asymptomatic

**trtbps** : resting blood pressure (in mm Hg)

**chol** : cholestoral in mg/dl fetched via BMI sensor

**fbs** : (fasting blood sugar > 120 mg/dl) 

> 1 = true

> 0 = false

**rest_ecg** : resting electrocardiographic results

> *Value 0*: normal

> *Value 1*: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)

> *Value 2*: showing probable or definite left ventricular hypertrophy by Estes' criteria

**thalach** : maximum heart rate achieved

**target** : 

> 0= less chance of heart attack 

> 1= more chance of heart attack

In [ ]:
heart_data.head()

In [ ]:
heart_data.output.unique()

In [ ]:
O2Saturation_data.head()

# **<span style="color:#FEB139;">🦾Prepare data for model</span>**

In [ ]:
X = heart_data.drop(columns = 'output')
y = heart_data['output']

In [ ]:
X.shape

In [ ]:
y.shape

# **<span style="color:#FEB139;">🪓Train Test Split</span>**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state=42)

# **<span style="color:#B25068;">MODEL 1 - Random Forest Classifier</span>**

In [ ]:
model_1 = RandomForestClassifier(n_estimators = 100)

model_1.fit(X_train, y_train)

In [ ]:
# Predictions
predictions_1 = model_1.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, predictions_1, labels = model_1.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = model_1.classes_)
disp.plot()

In [ ]:
clf_report = classification_report(y_test, predictions_1)
print(clf_report)

In [ ]:
# ROC Curve

y_pred_prob = model_1.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr)
plt.show()

In [ ]:
steps = [('model', RandomForestClassifier(n_estimators = 100))]
pipeline = Pipeline(steps=steps)

# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

# **<span style="color:#B25068;">MODEL 2 - XGBoost Classifier</span>**

In [ ]:
model_2 = XGBClassifier()

model_2.fit(X_train, y_train)

In [ ]:
# Predictions

predictions_2 = model_2.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, predictions_2, labels = model_2.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = model_2.classes_)
disp.plot()

In [ ]:
clf_report = classification_report(y_test, predictions_2)
print(clf_report)

In [ ]:
# ROC Curve

y_pred_prob = model_2.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr)
plt.show()

In [ ]:
steps = [('model', XGBClassifier())]
pipeline = Pipeline(steps=steps)

# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

## 🤘Conclusion

Random Forest Classifier did better with more area under the ROC curve than XGBoost Classifier.

**This is not end of 🩺♥Heart Attack Analysis & Prediction**

**Stay Tuned for more analysis on this dataset**

**Please share your feedback and suggestions and help me improve 😇**